### 두개의 CSV 파일 공통 키 기준 합치기

일별 집계된 주문 데이터 + 월별 경제심리 지수 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

### 1. 일집계 Data Read

In [2]:
po_df = pd.read_csv('./B72_7MA_14MA.csv')    # 일집계 주문데이터

In [3]:
po_df.head(10)

,OrderDate,YYYYWEEK,OrderQty,7MA,14MA
0,2018-01-02,201801,92,NaN,NaN
1,2018-01-03,201801,130,NaN,NaN
2,2018-01-04,201801,45,NaN,NaN
3,2018-01-05,201801,54,NaN,NaN
4,2018-01-08,201802,464,NaN,NaN
5,2018-01-09,201802,60,NaN,NaN
6,2018-01-10,201802,60,129.285714,NaN
7,2018-01-11,201802,50,123.285714,NaN
8,2018-01-12,201802,2,105.000000,NaN
9,2018-01-15,201803,85,110.714286,NaN


### 2. OrderDate를 잘라 YYYYMM 형태 변경하여 신규 컬럼 생성

In [4]:
po_df['YYYYMM'] = po_df['OrderDate'].str.slice(0, 4) + po_df['OrderDate'].str.slice(5, 7)

In [5]:
po_df.info()
#OrderDate 의 자료형은 object 타입으로 주단위 계산을 위해 날자 타입으로 변경

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   OrderDate  1250 non-null   object 
 1   YYYYWEEK   1250 non-null   int64  
 2   OrderQty   1250 non-null   int64  
 3   7MA        1244 non-null   float64
 4   14MA       1237 non-null   float64
 5   YYYYMM     1250 non-null   object 
dtypes: float64(2), int64(2), object(2)
memory usage: 58.7+ KB


### 3. 경제심리지수 Data Read

In [6]:
ps_df = pd.read_csv('./513Y001.csv', encoding='CP949')   #월별 경제심리 지수

In [8]:
ps_df.head(5)

,Unnamed: 0,STAT_CODE,STAT_NAME,ITEM_CODE1,ITEM_NAME1,ITEM_CODE2,ITEM_NAME2,ITEM_CODE3,ITEM_NAME3,ITEM_CODE4,ITEM_NAME4,UNIT_NAME,TIME,DATA_VALUE
0,0,513Y001,6.3. 경제심리지수,E1000,경제심리지수(원계열),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201701,95.9
1,1,513Y001,6.3. 경제심리지수,E1000,경제심리지수(원계열),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201702,97.7
2,2,513Y001,6.3. 경제심리지수,E1000,경제심리지수(원계열),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201703,100.0
3,3,513Y001,6.3. 경제심리지수,E1000,경제심리지수(원계열),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201704,99.6
4,4,513Y001,6.3. 경제심리지수,E1000,경제심리지수(원계열),NaN,NaN,NaN,NaN,NaN,NaN,NaN,201705,100.7


In [9]:
##필요 컬럼 추출 
ps_df = ps_df[['TIME', 'DATA_VALUE']]

### 4. po_df.['YYYYMM']과 ps_df.['TIME'] 의 자료형을 int타입으로 통일

In [10]:
ps_df['TIME'] = ps_df['TIME'].astype(str)   #오더번호는 string

In [12]:
ps_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TIME        69 non-null     object 
 1   DATA_VALUE  69 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.2+ KB


### 5. po_df 기준으로 left outer join, 

#### 주문테이블에 존재하는 데이터는 전부 표기 

#### 단 해당값은 Uniq 해야 조인이 가능

In [14]:
#http://www.gisdeveloper.co.kr/?p=8255
pops_df = pd.merge(po_df, ps_df, left_on='YYYYMM', right_on='TIME', how='left')

In [17]:
pops_df.head(5)

,OrderDate,YYYYWEEK,OrderQty,7MA,14MA,YYYYMM,TIME,DATA_VALUE
0,2018-01-02,201801,92,NaN,NaN,201801,201801,99.4
1,2018-01-03,201801,130,NaN,NaN,201801,201801,99.4
2,2018-01-04,201801,45,NaN,NaN,201801,201801,99.4
3,2018-01-05,201801,54,NaN,NaN,201801,201801,99.4
4,2018-01-08,201802,464,NaN,NaN,201801,201801,99.4
